In [ ]:
import tensorflow as tf
import numpy as np
import sys

from datetime import datetime

python_root = '../'
sys.path.insert(0, python_root)
from model.alexnet import AlexNet
from model.triplet_loss import batch_all_triplet_loss, batch_hardest_triplet_loss

In [ ]:
np.random.seed(1)

## Model

In [ ]:
## Configuration settings
import os
# Path to the textfiles for the trainings and validation set
train_file = '../train.txt'
val_file = '../val.txt'
test_file = '../test.txt'

# Learning params
learning_rate = 0.01
num_epochs = 30
batch_size = 128
margin = 0.2
triplet_strategy = "batch_all"
# triplet_strategy = "batch_hard"

# Network params
dropout_rate = 0.5
no_train_layers = []

# How often we want to write the tf.summary data to disk
display_step = 1

# Path for tf.summary.FileWriter and to store model checkpoints
filewriter_path = "../logs/tmp/triplet_loss_{}".format(triplet_strategy)
checkpoint_path = "../logs/tmp/triplet_loss_{}_ckpt".format(triplet_strategy)

# Create parent path if it doesn't exist
if not os.path.isdir(checkpoint_path): os.makedirs(checkpoint_path)

In [ ]:
# TF placeholder for graph input and output
x = tf.placeholder(tf.float32, [batch_size, 227, 227, 3])
labels = tf.placeholder(tf.float32, [batch_size, 4])
keep_prob = tf.placeholder(tf.float32)

# Initialize model
skip_layers = ['fc8']  # Don't use weights from AlexNet
model = AlexNet(x, keep_prob, skip_layers, '../model/bvlc_alexnet.npy')

# Link variable to model output
embeddings = tf.nn.l2_normalize(model.fc8, axis=1)

# List of trainable variables of the layers we want to train
var_list = [v for v in tf.trainable_variables() if v.name.split('/')[0] not in no_train_layers]

with tf.name_scope("triplet_loss"):
    if triplet_strategy == "batch_all":
        loss, fraction = batch_all_triplet_loss(labels, embeddings, margin=margin, squared=False)
    elif triplet_strategy == "batch_hard":
        loss = batch_hardest_triplet_loss(labels, embeddings, margin=margin, squared=False)
    else:
        raise ValueError("Triplet strategy not recognized: {}".format(triplet_strategy))

In [ ]:
# Train op
with tf.name_scope("train"):
  # Get gradients of all trainable variables
    gradients = tf.gradients(loss, var_list)
    gradients = list(zip(gradients, var_list))

    # Create optimizer and apply gradient descent to the trainable variables
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train_op = optimizer.apply_gradients(grads_and_vars=gradients)

In [ ]:
# Add gradients to summary
for gradient, var in gradients:
    tf.summary.histogram(var.name + '/gradient', gradient)

# Add the variables we train to the summary
for var in var_list:
    tf.summary.histogram(var.name, var)

# Add the loss to summary
if triplet_strategy == "batch_all":
    tf.summary.scalar('triplet_loss', loss)
    tf.summary.scalar('fraction_positive_triplets', fraction)
elif triplet_strategy == "batch_hard":
    tf.summary.scalar('triplet_loss', loss)

# Add embedding_mean_norm to summary
embedding_mean_norm = tf.reduce_mean(tf.norm(embeddings, axis=1))
tf.summary.scalar("embedding_mean_norm", embedding_mean_norm)

merged_summary = tf.summary.merge_all()

# Initialize the FileWriter
writer = tf.summary.FileWriter(filewriter_path)

# Initialize an saver for store model checkpoints
saver = tf.train.Saver()

In [ ]:
from model.datagenerator import ImageDataGenerator
train_generator = ImageDataGenerator(train_file,
                                     horizontal_flip = False, shuffle = True)
val_generator = ImageDataGenerator(val_file, shuffle = False)
test_generator = ImageDataGenerator(test_file, shuffle = False)

In [ ]:
# Get the number of training/validation steps per epoch
train_batches_per_epoch = np.floor(train_generator.data_size / batch_size).astype(np.int16)
val_batches_per_epoch = np.floor(val_generator.data_size / batch_size).astype(np.int16)

# Start training

You can also skip training and use the checkpoint at 30 epochs in `../logs/model_epoch30.ckpt`. See [Embeddings](#Embeddings).

In [ ]:
with tf.Session() as sess:
    # Initialize all variables
    sess.run(tf.global_variables_initializer())
    
    # Add the model graph to TensorBoard
    writer.add_graph(sess.graph)
    
    # Load the pretrained weights into the non-trainable layer
    model.load_initial_weights(sess)
    
    print("{} Start training...".format(datetime.now()))
    print("{} Open Tensorboard at --logdir {}".format(datetime.now(), filewriter_path))
    
    # Loop over number of epochs
    for epoch in range(num_epochs):

        print("{} Epoch number: {}".format(datetime.now(), epoch+1))

        step = 1

        while step < train_batches_per_epoch:

            # Get a batch of images and labels
            batch_x_train, batch_labels_train = train_generator.next_batch(batch_size)

            # And run the training op
            sess.run(train_op, feed_dict={x: batch_x_train,
                                          labels: batch_labels_train,
                                          keep_prob: dropout_rate})

            # Generate summary with the current batch of data and write to file
            if step%display_step == 0:
                s = sess.run(merged_summary, feed_dict={x: batch_x_train,
                                                        labels: batch_labels_train,
                                                        keep_prob: 1.})
                writer.add_summary(s, epoch*train_batches_per_epoch + step)

            step += 1

        # Validate the model on the entire validation set
        print("{} Start validation".format(datetime.now()))
        loss_val = 0.
        test_count = 0
        for _ in range(val_batches_per_epoch):
            batch_tx, batch_ty = val_generator.next_batch(batch_size)
            loss_current = sess.run(loss, feed_dict={x: batch_tx,
                                                     labels: batch_ty,
                                                     keep_prob: 1.})
            loss_val += loss_current
            test_count += 1
        loss_val = loss_val / test_count
        print("{} Average loss for validation set = {:.4f}".format(datetime.now(), loss_val))
        
        # Reset the file pointer of the image data generator
        val_generator.reset_pointer()
        train_generator.reset_pointer()
        
        print("{} Saving checkpoint of model...".format(datetime.now()))
        #save checkpoint of the model
        checkpoint_name = os.path.join(checkpoint_path, 'model_epoch'+str(epoch+1)+'.ckpt')
        save_path = saver.save(sess, checkpoint_name)

        print("{} Model checkpoint saved at {}".format(datetime.now(), checkpoint_name))


# Embeddings

## Get embeddings of lines in test set

In [ ]:
# According to ../split_dataset_with_labels_world.py
test = []
traj = 1
frames_total = 300
for frame_id in range(frames_total):
    if frame_id % 5 == 3:
        test.append(frame_id)
        continue

test_set_size = 0
with open(test_file) as f:
    for i, l in enumerate(f):
        pass
    test_set_size = i + 1 
    
assert len(test) != test_set_size
print("Test set has {} lines ".format(test_set_size))

In [ ]:
%%time
test_embeddings_all = np.empty((0, 64), dtype=np.float32)

with tf.Session() as sess:
    saver.restore(sess, '../logs/model_epoch30.ckpt')
    
    test_generator.reset_pointer()
    
    for i in range(test_set_size / batch_size):
        batch_x, batch_labels = test_generator.next_batch(batch_size)
        output = sess.run(embeddings, feed_dict={x: batch_x,
                                                 labels: batch_labels,
                                                 keep_prob: 1.})
        test_embeddings_all = np.vstack([test_embeddings_all, output])
        # Display every 5 steps
        if i%5 == 0:
            print("Embeddings got for {} lines ".format(test_embeddings_all.shape[0]))
            
    lines_totoal = test_embeddings_all.shape[0]
    print("Embeddings got for {} lines ".format(lines_totoal))

# K-means clustering with obtained embeddings

In [ ]:
from sklearn.cluster import KMeans

lines_features = test_embeddings_all

n_clusters = 30
kmeans = KMeans(n_clusters, init='k-means++').fit(lines_features)
cluster_labels = kmeans.labels_

In [ ]:
from tools.visualization import get_lines_world_coordinates_with_instances

data_lines_world = get_lines_world_coordinates_with_instances(trajectory=traj, frames=test)
data_lines_world = data_lines_world[:lines_totoal]

In [ ]:
from tools.scenenet_utils import pcl_lines_for_plot

pcl_lines_open3d = pcl_lines_for_plot(data_lines_world, lines_color=cluster_labels)

In [ ]:
import open3d

open3d.draw_geometries(pcl_lines_open3d[:])

## See embeddings in the feature space, colored with instance label

In [ ]:
from tensorflow.contrib.tensorboard.plugins import projector

LOG_DIR = os.path.abspath('../logs/tmp/embedding_logs')
# Create parent path if it doesn't exist
if not os.path.isdir(LOG_DIR): os.makedirs(LOG_DIR)
    
metadata = os.path.join(LOG_DIR, 'embedding_metadata.tsv')

test_embeddings = tf.Variable(test_embeddings_all, name='test_embeddings')

with open(metadata, 'w') as metadata_file:
    for label in cluster_labels:
        metadata_file.write('%d\n' % label)
        
with tf.Session() as sess:
    saver = tf.train.Saver([test_embeddings])

    sess.run(test_embeddings.initializer)
    saver.save(sess, os.path.join(LOG_DIR, 'test_embeddings.ckpt'))

    config = projector.ProjectorConfig()
    # One can add multiple embeddings.
    embedding = config.embeddings.add()
    embedding.tensor_name = test_embeddings.name
    # Link this tensor to its metadata file (e.g. labels).
    embedding.metadata_path = metadata
    # Saves a config file that TensorBoard will read during startup.
    projector.visualize_embeddings(tf.summary.FileWriter(LOG_DIR), config)

In [ ]:
!tensorboard --logdir ../logs/tmp/embedding_logs